In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from time import gmtime, strftime

In [8]:
df_train = pd.read_csv('train_V2.csv')
df_test = pd.read_csv('test_V2.csv')

In [9]:
df_rand = df_train.reindex(np.random.permutation(df_train.index))

In [10]:
df_rand2 = df_train.reindex()
#df_rand2.head()

In [11]:
df_rand3 = df_train.reindex(np.random.permutation(df_train.shape[0]))
df_rand3.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
714128,2c1cc480990920,70d4154d5216db,195c5bd37a85dc,0,0,0.00,0,0,0,74,...,0,0.0,0,0.0,1,0,784.7,4,0,0.2222
199973,c989c734b439d1,a8f84ddd3ee9eb,5845919df05838,0,0,233.60,3,0,0,26,...,0,0.0,0,0.0,0,0,292.0,1,0,0.3200
4105138,ce86cce2206073,04748c80d36d1f,a7e407dde9f3db,0,0,0.00,0,0,0,55,...,0,0.0,0,0.0,0,0,1938.0,4,0,0.5517
3747925,ef75f2f1595316,85aab146f0ec7a,959aa80e79df42,2,5,254.80,1,1,2,28,...,1,2282.0,0,0.0,0,0,4361.0,8,1498,0.7931
2593936,652456f8940af4,3aec0e67ea627d,e23346e33ac4c9,0,0,33.58,0,0,0,89,...,0,0.0,0,0.0,0,0,106.1,2,1446,0.0345


### Preprocess the data for training, validation and testing

In [12]:
X_train = df_rand3[['killPlace', 'walkDistance']].head(4000000)
Y_train = df_rand3[['winPlacePerc']].head(4000000)
X_val = df_rand3[['killPlace', 'walkDistance']].tail(446966)
Y_val = df_rand3[['winPlacePerc']].tail(446966)
X_test = df_test[['killPlace', 'walkDistance']]

### Create the feature columns

In [13]:
#help(tf.feature_column.numeric_column)

In [15]:
killPlace_feat_col = tf.feature_column.numeric_column(key='killPlace')
walkDist_feat_col = tf.feature_column.numeric_column(key='walkDistance')
feat_cols = set([killPlace_feat_col, walkDist_feat_col])

In [16]:
feat_cols

{NumericColumn(key='killPlace', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='walkDistance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)}

### Input Functions

In [17]:
# Misc tests and learnings:
#X_train.head()
#X_np_train[:,0], is the correct way to access column values in an np array
#.values returns an np nd array from a DataFrame
#help(np.array)

In [32]:
#help(tf.estimator.inputs.numpy_input_fn)

numpy_input_fn
There is only 2 arguments that are neccesary
    1) x: takes a dictionary with keys as the column names and values as the column values
    2) shuffle
Returns:
    An input function that would feed a dictionary of numpy arrays into the model

In [29]:
# Train input func
# convert to array
X_np_train = np.array(X_train)
x_train = {'killPlace': X_np_train[:,0], 'walkDistance': X_np_train[:,1]}
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_train,
    y=np.array(Y_train),
    num_epochs=None,
    shuffle=True)
#num_epochs determines what? the passes that I do along the data?
# if we set num_epochs to none it allows us to read as much data as we can from this input fn?
# shuffle randomizes the data

In [33]:
# train_input_fn()
# We can see that it is a function that returns a an input function 
# type(train_input_fn)

In [34]:
# Validation input func
#X_val_np = X_val.values
x_val_in = {'killPlace': X_val.values[:,0], 'walkDistance': X_val.values[:,1]}
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_val_in,
    y=np.array(Y_val),
    num_epochs=1,
    shuffle=False)

In [40]:
# Prediction input function
x_test_in = {'killPlace': X_test.values[:,0], 'walkDistance': X_test.values[:,1]}
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_test_in,
    shuffle=False)

In [41]:
#help(tf.estimator.DNNRegressor)

In [42]:
# Timestamp for log folder
s = strftime("%a_%d_%b_%Y_%H:%M", gmtime())

In [43]:
# Create the Regressor Object of the tf.estimator.DNNRegressor Class
regressor = tf.estimator.DNNRegressor(
    feature_columns=[killPlace_feat_col, walkDist_feat_col],
    hidden_units=[256],
    model_dir='./logs/'+s,
    optimizer=tf.train.ProximalAdagradOptimizer(
        learning_rate=0.1,
        l1_regularization_strength=0.001
    )
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './logs/Sun_10_Mar_2019_23:52', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x107ade550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [46]:
# Train
print_periods = 10
#for i in range(print_periods)
regressor.train(
    input_fn=train_input_fn,
    steps=1000
)
metrics = regressor.evaluate(
        input_fn=val_input_fn,
    )
print(metrics)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/Sun_10_Mar_2019_23:52/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2000 into ./logs/Sun_10_Mar_2019_23:52/model.ckpt.
INFO:tensorflow:loss = 10.897235, step = 2001
INFO:tensorflow:global_step/sec: 470.983
INFO:tensorflow:loss = 10.401742, step = 2101 (0.213 sec)
INFO:tensorflow:global_step/sec: 644.732
INFO:tensorflow:loss = 9.979822, step = 2201 (0.155 sec)
INFO:tensorflow:global_step/sec: 561.675
INFO:tensorflow:loss = 10.224878, step = 2301 (0.178 sec)
INFO:tensorflow:global_step/sec: 577.654
INFO:tensorflow:loss = 11.627276, step = 2401 (0.174 sec)
INFO:tensorflow:global_step/sec: 722.611
INFO:tensorflow:loss = 10.733566, step = 2501 (0.138 sec)
INFO:tensorflow:global_step/sec: 627.738
INFO:

In [ ]:
#help(tf.estimator.DNNRegressor.train)

In [49]:
regressor.train(
    input_fn=train_input_fn,
    steps=1000
)
metrics = regressor.evaluate(
        input_fn=val_input_fn,
        steps=2000
    )
print(metrics)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./logs/Sun_10_Mar_2019_23:52/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4000 into ./logs/Sun_10_Mar_2019_23:52/model.ckpt.
INFO:tensorflow:loss = 7.7701907, step = 4001
INFO:tensorflow:global_step/sec: 622.956
INFO:tensorflow:loss = 11.164057, step = 4101 (0.161 sec)
INFO:tensorflow:global_step/sec: 846.913
INFO:tensorflow:loss = 8.495792, step = 4201 (0.118 sec)
INFO:tensorflow:global_step/sec: 922.101
INFO:tensorflow:loss = 8.732678, step = 4301 (0.108 sec)
INFO:tensorflow:global_step/sec: 982.077
INFO:tensorflow:loss = 5.232332, step = 4401 (0.103 sec)
INFO:tensorflow:global_step/sec: 694.503
INFO:tensorflow:loss = 9.369472, step = 4501 (0.143 sec)
INFO:tensorflow:global_step/sec: 759.047
INFO:ten